# **Advanced Lane Finding** 

---
By Hasan Korre

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply the distortion correction to the raw image.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find lane boundary.
* Determine curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---

## Camera Calibration (Distortion Correction)

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

# Read in a calibration image
img = mpimg.imread('camera_cal/calibration2.jpg')
plt.imshow(img)

### Corner Detection for 1 image

In [ ]:
# Arrays to store object points and image points from all the images
obj_points = []  #3D points in real world space
img_points = []  #2D point in image plane

# Prepare object points, like (0,0,0), (1,0,0), (2,0,0), (8,5,0)
nx = 9  #num of inside corners in x
ny = 6  #num of inside corners in y
objp = np.zeros((ny*nx,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Convert image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

# Find the chessboard corners
ret, corners = cv2.findChessboardCorners(gray,(nx, ny), None)

# If corners are found, add object points and image points
if ret == True:
    img_points.append(corners)
    obj_points.append(objp)
    
    # Draw and display the corners
    cv2.drawChessboardCorners(img, (nx, ny), corners, ret)
    plt.imshow(img)   

### Calibrate the camera

In [ ]:
import glob

# Read in and make a list of calibration image
images = glob.glob('camera_cal/calibration*.jpg')

# Arrays to store object points and image points from all the images
obj_points = []  #3D points in real world space
img_points = []  #2D point in image plane

# Prepare object points, like (0,0,0), (1,0,0), (2,0,0), (8,5,0)
nx = 9  #num of inside corners in x
ny = 6  #num of inside corners in y
objp = np.zeros((ny*nx,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

for filename in images:
    # read in each image
    img = mpimg.imread(filename)
    
    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray,(nx, ny), None)

    # If corners are found, add object points and image points
    if ret == True:
        img_points.append(corners)
        obj_points.append(objp)

# Calibrate the camera
img_size = (img.shape[1], img.shape[0])
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points, img_points, img_size, None, None)

print('Success: Calibrated camera.')

### Undistort an image

In [ ]:
# Undistortion function
#   mtx_  = camera matrix
#   dist_ = distortion coefficients
def cal_undistort(img_, mtx_, dist_):
    return cv2.undistort(img_, mtx_, dist_, None, mtx_)

# Test
img_original = mpimg.imread('camera_cal/calibration5.jpg')
img_undistort = cal_undistort(img_original, mtx, dist)

# Display
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img_original)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(img_undistort)
ax2.set_title('Undistorted Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

## Create a thresholded binary image

### Color transforms and SobelX

In [ ]:
# Convert to HLS color space and separate the S channel
def rgb_to_s(img_):   
    hls = cv2.cvtColor(img_, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    return s_channel

# RBG to Grayscale
def rbg_to_gray(img_):
    return cv2.cvtColor(img_, cv2.COLOR_RGB2GRAY)

# Get X gradient through sobelx operator
# ksize – size of the extended Sobel kernel; it must be 1, 3, 5, or 7
def sobelx(img_, kernel_=3):
    gray = rbg_to_gray(img_)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=kernel_) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx)
    #rescaling to 8-bit
    scale_factor = np.max(abs_sobelx)/255
    return (abs_sobelx/scale_factor).astype(np.uint8)

print('Success: Color transform and SobelX functions defined.')

In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

# Example Color Transform
test_img = mpimg.imread('test_images/test4.jpg')
test_sChannel = rgb_to_s(test_img)

# Display
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(test_img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(test_sChannel, cmap='gray')
ax2.set_title('S-Channel Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
%matplotlib inline

# Example Sobelx Transform
test_img = mpimg.imread('test_images/test4.jpg')
test_sobelx = sobelx(test_img, 3)

# Display
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(test_img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(test_sobelx, cmap='gray')
ax2.set_title('SobelX Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

### Thresholding

In [ ]:
# Threshold color channel
def threshold_color(img_channel_, s_thresh_min_, s_thresh_max_):
    s_binary = np.zeros_like(img_channel_)
    s_binary[(img_channel_ >= s_thresh_min_) & (img_channel_ <= s_thresh_max_)] = 1
    return s_binary

# Threshold x gradient
def threshold_xGradient(img_sobel_, thresh_min_, thresh_max_):
    sxbinary = np.zeros_like(img_sobel_)
    sxbinary[(img_sobel_ >= thresh_min_) & (img_sobel_ <= thresh_max_)] = 1
    return sxbinary

print('Success: Thresholding functions defined.')

In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

# Example Color Thresholding
test_img = mpimg.imread('test_images/test4.jpg')
test_sChannel = rgb_to_s(test_img)
test_color_thresh = threshold_color(test_sChannel, 170, 255)

# Display
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(test_sChannel, cmap='gray')
ax1.set_title('S-Channel Image', fontsize=50)
ax2.imshow(test_color_thresh, cmap='gray')
ax2.set_title('Thresholded S-Channel Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

# Example X-Gradient Thresholding
test_img = mpimg.imread('test_images/test4.jpg')
test_sobelx = sobelx(test_img, 3)
test_sobelx_thresh = threshold_xGradient(test_sobelx, 20, 100)

# Display
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(test_sobelx, cmap='gray')
ax1.set_title('SobelX Image', fontsize=50)
ax2.imshow(test_sobelx_thresh, cmap='gray')
ax2.set_title('Thresholded SobelX Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

### Sobel and Color Thresholding Combined

In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
%matplotlib inline

def combine_threshold(img_):
    # Color Thresholding
    sChannel_img = rgb_to_s(img_)
    color_thresh = threshold_color(sChannel_img, 170, 255)
    
    # Example X-Gradient Thresholding
    sobelx_img = sobelx(img_, 3)
    sobelx_thresh = threshold_xGradient(sobelx_img, 20, 100)
    
    # Stack each channel to view their individual contributions in green and blue respectively
    # This returns a stack of the two binary images, whose components you can see as different colors
    color_binary = np.dstack(( np.zeros_like(sobelx_thresh),
                               sobelx_thresh,
                               color_thresh ))
    color_binary[color_binary > 0.5] = 255  #help plotting of rgb
    
    # Combine the two binary thresholds
    combined_binary = np.zeros_like(sobelx_thresh)
    combined_binary[(sobelx_thresh == 1) | (color_thresh == 1)] = 1
    
    return color_binary, combined_binary


test_img = mpimg.imread('test_images/test4.jpg')
color_binary, combined_binary = combine_threshold(test_img)


# Plotting thresholded images
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.set_title('Stacked thresholds (green=sobelx, blue=s_channel)')
ax1.imshow(color_binary)

ax2.set_title('Combined S channel and gradient thresholds')
ax2.imshow(combined_binary, cmap='gray')


## Perspective Transform

In [ ]:
import numpy as np

warp_src = np.float32([[0,0],[0,0],[0,0],[0,0]])

warp_dst = np.float32([[0,0],[0,0],[0,0],[0,0]])

X_INDEX = 0
Y_INDEX = 1

def update_warp_points(img_):
    global warp_src
    global warp_dst
    
    height = img_.shape[0]
    width  = img_.shape[1]
        
    #top_left
    warp_src[0,X_INDEX] = width*0.4725 
    warp_src[0,Y_INDEX] = height*0.60  
    #top_right
    warp_src[1,X_INDEX] = width*0.53 
    warp_src[1,Y_INDEX] = height*0.60
    #bottom_left
    warp_src[2,X_INDEX] = width*0.17 
    warp_src[2,Y_INDEX] = height*0.99
    #bottom_right
    warp_src[3,X_INDEX] = width*0.875
    warp_src[3,Y_INDEX] = height*0.99

    #top_left
    warp_dst[0,X_INDEX] = width*0.35 
    warp_dst[0,Y_INDEX] = height*0.01    
    #top_right
    warp_dst[1,X_INDEX] = width*0.65 
    warp_dst[1,Y_INDEX] = height*0.01
    #bottom_left
    warp_dst[2,X_INDEX] = width*0.35 
    warp_dst[2,Y_INDEX] = height*0.99
    #bottom_right
    warp_dst[3,X_INDEX] = width*0.65 
    warp_dst[3,Y_INDEX] = height*0.99
    
    
def perspective_warp(img_):
    update_warp_points(img_)
        
    img_size = (img_.shape[1], img_.shape[0])
    M = cv2.getPerspectiveTransform(warp_src, warp_dst)
    return cv2.warpPerspective(img_, M, img_size, flags=cv2.INTER_LINEAR)

def perspective_warp_inv(img_):
    update_warp_points(img_)
    
    img_size = (img_.shape[1], img_.shape[0])
    Minv = cv2.getPerspectiveTransform(warp_dst, warp_src)
    return cv2.warpPerspective(img_, Minv, img_size, flags=cv2.INTER_LINEAR)

print('Success: Perspective Transform functions defined.')

In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

# Example of Perspective Transform
test_img = mpimg.imread('test_images/solidWhiteRight.jpg')
#test_img = mpimg.imread('test_images/test5.jpg')
warped = perspective_warp(test_img)


# Plotting transformed image
f, (ax1, ax2) = plt.subplots(2, 1, figsize=(20,10))
ax1.set_title('Original: Marked Src Points')
ax1.imshow(test_img)

ax1.plot(warp_src[0,0],warp_src[0,1],'.')  #top left
ax1.plot(warp_src[1,0],warp_src[1,1],'.')  #top right
ax1.plot(warp_src[2,0],warp_src[2,1],'.')  #bottom left
ax1.plot(warp_src[3,0],warp_src[3,1],'.')  #bottom right


ax2.set_title('Perspective Transform: Marked Dst Points')
ax2.imshow(warped)
ax2.plot(warp_dst[0,0],warp_dst[0,1],'.')  #top left
ax2.plot(warp_dst[1,0],warp_dst[1,1],'.')  #top right
ax2.plot(warp_dst[2,0],warp_dst[2,1],'.')  #bottom left
ax2.plot(warp_dst[3,0],warp_dst[3,1],'.')  #bottom right


## Finding Curves

### Region of Interest

In [ ]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

"""
Region of Interest:
Goal: Only keep marker that are in the center
"""

def region_of_interest(img_, vertices_):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img_)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img_.shape) > 2:
        channel_count = img_.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices_, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img_, mask)
    return masked_image


def hk_region_ofInterest(img_):
    height = img_.shape[0]
    width  = img_.shape[1]
    
    height_mult  = 0.60
    width_mult_R = 0.54
    width_mult_L = 0.44
    
    # [horiz, vert]
    top_left     = [width*width_mult_L, height*height_mult]
    top_right    = [width*width_mult_R, height*height_mult]
    bottom_right = [width, height]
    bottom_left  = [0, height]
    
    poly = np.array([top_left, top_right, bottom_right, bottom_left], np.int32)
    return region_of_interest(img_, [poly])

'''
run it...
'''  
test_img = mpimg.imread('test_images/test5.jpg')

# transforms
_, combined_binary = combine_threshold(test_img)
test_img_roi = hk_region_ofInterest(combined_binary)

# Display the image
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.set_title('Original Image')
ax1.imshow(test_img)

ax2.set_title('Region of Interest')
ax2.imshow(test_img_roi, cmap='gray')

### Find points on line

In [ ]:
import numpy as np

#test_img = mpimg.imread('test_images/solidWhiteRight.jpg')
test_img = mpimg.imread('test_images/test5.jpg')

# transforms
_, combined_binary = combine_threshold(test_img)
test_img_roi = hk_region_ofInterest(combined_binary)
test_warped = perspective_warp(test_img_roi)

NUM_BANDS = 10
TRACK_THRESHOLD = 20
BOX_HALF_WIDTH = 50

def find_point(histogram_, avg_height_, is_initialized_, min_edge_, max_edge_, last_center_, list_):
    # decide width to look in for max
    if is_initialized_ == True:
        left_edge = last_center_ - BOX_HALF_WIDTH
        right_edge = last_center_ + BOX_HALF_WIDTH            
    else:
        left_edge = min_edge_
        right_edge = max_edge_

    #print('\t(L_edge,R_edge)=({},{})'.format(left_edge, right_edge))
    # find the max
    center_guess = np.argmax(histogram_[left_edge:right_edge])+left_edge
    center_value = histogram_[center_guess]
    #print('\tcenter_guess: ', center_guess)
    #print('\tcenter_value: ', center_value)
    if center_value > TRACK_THRESHOLD:
        #print('\tappending...')
        list_.append((center_guess, avg_height_))
        last_center_ = center_guess
        if is_initialized_ == False:
            is_initialized_ = True

    return is_initialized_, last_center_, list_


def get_lane_points(img_):
    left = []
    right = []
    

    #band_start = img_.shape[0]
    band_depth = int(img_.shape[0]/NUM_BANDS)
    
    tops = np.arange(0, img_.shape[0]-1, band_depth)
    bottoms = tops + (band_depth-1)
    
    # reverse the arrays
    tops = tops[::-1]
    bottoms = bottoms[::-1]
    
    is_left_initialized = False
    is_right_initialized = False
    last_left_center = 0
    last_right_center = 0
    
    for index in range(len(tops)):
        #print('(top, bottom)=({},{})'.format(tops[index], bottoms[index]))
        histogram = np.sum(img_[tops[index]:bottoms[index],:], axis=0)
        hist_len = histogram.shape[0]
        avg_height = (tops[index] + bottoms[index])/2

        is_left_initialized, last_left_center, left = find_point(histogram,
                                                                 avg_height,
                                                                 is_left_initialized, 
                                                                 0, 
                                                                 int(hist_len/2), 
                                                                 last_left_center, 
                                                                 left)
        
        is_right_initialized, last_right_center, right = find_point(histogram,
                                                                 avg_height,
                                                                 is_right_initialized, 
                                                                 int(hist_len/2), 
                                                                 hist_len-1, 
                                                                 last_right_center, 
                                                                 right)
            
    #print(left)
    
    return left, right
    

left, right = get_lane_points(test_warped)

def add_points(fig_, points_, marking_):
    for pt_tuple in points_:
        fig_.plot(pt_tuple[0],pt_tuple[1],marking_, markersize=30)



f, (ax1) = plt.subplots(1, 1, figsize=(20,10))
ax1.set_title('Original: Marked Lane Points')
ax1.imshow(test_warped, cmap='gray')
add_points(ax1, left, 'b.')
add_points(ax1, right, 'r.')

### Curve Fitting

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline


def fit_curve(points_):
    # get x and y values
    x_vals = np.asarray([pt_tuple[0] for pt_tuple in points_])
    y_vals = np.asarray([pt_tuple[1] for pt_tuple in points_])
    # Fit a second order polynomial
    fit_coeff = np.polyfit(y_vals, x_vals, 2)
    x_fit = fit_coeff[0]*y_vals**2 + fit_coeff[1]*y_vals + fit_coeff[2]
    return x_fit, y_vals, fit_coeff

def add_curve(fig_, x_fit_, y_vals_):
    fig_.plot(x_fit_, y_vals_, color='green', linewidth=3)


# grab image
test_img = mpimg.imread('test_images/test5.jpg')

# transforms
_, combined_binary = combine_threshold(test_img)
test_img_roi = hk_region_ofInterest(combined_binary)
test_warped = perspective_warp(test_img_roi)

# find the curve
left, right = get_lane_points(test_warped)
left_x_fit, left_y_vals, _ = fit_curve(left)
right_x_fit, right_y_vals, _ = fit_curve(right)

f, (ax1) = plt.subplots(1, 1, figsize=(20,10))
ax1.set_title('Original: Marked Lane Points')
ax1.imshow(test_warped, cmap='gray')
add_points(ax1, left, 'b.')
add_points(ax1, right, 'r.')
add_curve(ax1, left_x_fit, left_y_vals)
add_curve(ax1, right_x_fit, right_y_vals)

### Path Overlay

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline


def path_overlay(rgb_img_, left_fitx_, left_yvals_, right_fitx_, right_yvals_):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(rgb_img_[:,:,0]).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx_, left_yvals_]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx_, right_yvals_])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    return cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

def gen_points_from_fit(num_points_, height_, y_min_, fit_coeff_):
    y_gen = np.linspace(y_min_, height_-1, num=num_points_)
    x_gen = fit_coeff_[0]*y_gen**2 + fit_coeff_[1]*y_gen + fit_coeff_[2]
    return x_gen, y_gen
    


# grab image
test_img = mpimg.imread('test_images/test5.jpg')

# transforms
_, combined_binary = combine_threshold(test_img)
test_img_roi = hk_region_ofInterest(combined_binary)
test_warped = perspective_warp(test_img_roi)

# find the curve
left, right = get_lane_points(test_warped)
left_x_fit, left_y_vals, left_fit_coeff = fit_curve(left)
right_x_fit, right_y_vals, right_fit_coeff = fit_curve(right)

test_img_warped = perspective_warp(test_img)

# create more points for the overlay
NUM_OVERLAY_PTS = 10
test_height = test_img.shape[0]
left_x_gen, left_y_gen = gen_points_from_fit(NUM_OVERLAY_PTS, test_height, left_y_vals[-1], left_fit_coeff)
right_x_gen, right_y_gen = gen_points_from_fit(NUM_OVERLAY_PTS, test_height, right_y_vals[-1], right_fit_coeff)

# generate overlays
warped_overlay = path_overlay(test_img_warped, left_x_gen, left_y_gen, right_x_gen, right_y_gen)
unwarped_overlay = perspective_warp_inv(warped_overlay)

# put overlays on images
testImg_warp_overlay = cv2.addWeighted(test_img_warped, 1, warped_overlay, 0.3, 0)
testImg_unwarp_overlay = cv2.addWeighted(test_img, 1, unwarped_overlay, 0.3, 0)


f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.set_title('Overlay in Warped Image')
ax1.imshow(testImg_warp_overlay)

ax2.set_title('Overlay in Original Image')
ax2.imshow(testImg_unwarp_overlay)
